In [ ]:
import os
import pandas as pd
import numpy as np
from scipy.io.wavfile import write
from tqdm import tqdm

# Ensure directories exist
os.makedirs('outputs/10_percent_samples', exist_ok=True)
os.makedirs('outputs/60_second_tracks', exist_ok=True)

# Read the EEG data from a CSV file
data = pd.read_csv('data/eeg_data.csv', low_memory=False)
data['TimeStamp'] = pd.to_datetime(data['TimeStamp'])
data.set_index('TimeStamp', inplace=True)

# Combine EEG band columns into a single column per band
data['Delta'] = data[['Delta_TP9', 'Delta_AF7', 'Delta_AF8', 'Delta_TP10']].sum(axis=1)
data['Theta'] = data[['Theta_TP9', 'Theta_AF7', 'Theta_AF8', 'Theta_TP10']].sum(axis=1)
data['Alpha'] = data[['Alpha_TP9', 'Alpha_AF7', 'Alpha_AF8', 'Alpha_TP10']].sum(axis=1)
data['Beta'] = data[['Beta_TP9', 'Beta_AF7', 'Beta_AF8', 'Beta_TP10']].sum(axis=1)
data['Gamma'] = data[['Gamma_TP9', 'Gamma_AF7', 'Gamma_AF8', 'Gamma_TP10']].sum(axis=1)

# Calculate the total recording duration from timestamps
timestamps = pd.to_datetime(data.index)
total_duration = (timestamps.max() - timestamps.min()).total_seconds()
print(f"Total Recording Duration: {total_duration} seconds")

# Sample duration in seconds
sample_duration = 10  # Adjust this to another number if needed
full_duration = 60  # Length for full tracks

# Constants
sample_rate = 44100  # Sample rate for audio

# Define EEG bands and their frequency ranges and modes
band_to_frequency_and_mode = {
    'Delta': {
        'freq_range': (65.41, 146.83),  # C2 to F#3
        'mode': [65.41, 73.42, 82.41, 97.99, 110.00, 130.81, 146.83],  # D minor
    },
    'Theta': {
        'freq_range': (196.00, 392.00),  # G3 to G4
        'mode': [196.00, 220.00, 246.94, 277.18, 329.63, 369.99, 392.00],  # A Dorian
    },
    'Alpha': {
        'freq_range': (261.63, 523.25),  # C4 to C5
        'mode': [261.63, 293.66, 329.63, 349.23, 392.00, 440.00, 493.88],  # C major
    },
    'Beta': {
        'freq_range': (329.63, 659.26),  # E4 to E5
        'mode': [329.63, 349.23, 392.00, 440.00, 493.88, 523.25, 587.33],  # E harmonic minor
    },
    'Gamma': {
        'freq_range': (880.00, 1760.00),  # A5 to A6
        'mode': [880.00, 987.77, 1108.73, 1244.51, 1396.91, 1567.98, 1760.00],  # F whole-tone
    }
}

# Helper function: Generate audio for each type
def generate_audio(data, freq_range, output_folder, audio_type, channel_name, duration):
    audio_type_names = {
        "synth": "Harmonic_Flow",
        "wave": "Resonant_Frequencies",
        "ecg": "Cosmic_Pulse"
    }
    mystical_name = audio_type_names[audio_type]
    print(f"Generating {mystical_name} for {channel_name} in {output_folder}...")
    os.makedirs(output_folder, exist_ok=True)
    data_min, data_max = data.min(), data.max()
    mapped_frequencies = np.interp(data, (data_min, data_max), freq_range)
    volume_factor = np.abs(np.sin(mapped_frequencies / max(freq_range)))
    audio_data = np.zeros(int(sample_rate * duration))
    time_values = np.linspace(0, duration, int(sample_rate * duration))

    if audio_type == "synth":
        with tqdm(total=len(mapped_frequencies), desc=f"{mystical_name} Progress") as pbar:
            for i, freq in enumerate(mapped_frequencies):
                audio_data += volume_factor[i % len(volume_factor)] * np.sin(2 * np.pi * freq * time_values)
                pbar.update(1)

    elif audio_type == "wave":
        revised_audio_data = np.zeros(int(sample_rate * duration))
        with tqdm(total=len(mapped_frequencies), desc=f"{mystical_name} Progress") as pbar:
            for i, freq in enumerate(mapped_frequencies):
                start_time = int(i * (duration / len(mapped_frequencies)) * sample_rate)
                t = np.linspace(0, duration / len(mapped_frequencies), int(sample_rate * (duration / len(mapped_frequencies))), endpoint=False)
                revised_audio_data[start_time:start_time + len(t)] += volume_factor[i % len(volume_factor)] * np.sin(2 * np.pi * freq * t)
                pbar.update(1)
        audio_data = revised_audio_data

    elif audio_type == "ecg":
        ecg_audio_data = np.zeros(int(sample_rate * duration))
        with tqdm(total=len(mapped_frequencies), desc=f"{mystical_name} Progress") as pbar:
            for i, freq in enumerate(mapped_frequencies):
                start_time = int(i * (duration / len(mapped_frequencies)) * sample_rate)
                t = np.linspace(0, duration / len(mapped_frequencies), int(sample_rate * (duration / len(mapped_frequencies))), endpoint=False)
                base_freq = freq * 0.6  # Simulate bass-like sound
                heartbeat_freq = 1.0  # Heartbeat frequency (1 Hz)
                
                # Smooth pulse: sinusoidal envelope
                pulse = 0.5 * (1 + np.sin(2 * np.pi * heartbeat_freq * t))  # Sinusoidal pulse
                pulse = np.clip(pulse, 0, 1)  # Ensure the pulse stays positive
                
                # Create the heartbeat sound
                heartbeat = pulse * np.sin(2 * np.pi * base_freq * t)
                
                # Add fade-in and fade-out for smoother transitions
                fade_length = int(0.1 * len(t))  # Fade length is 10% of the duration
                fade_in = np.linspace(0, 1, fade_length)
                fade_out = np.linspace(1, 0, fade_length)
                heartbeat[:fade_length] *= fade_in
                heartbeat[-fade_length:] *= fade_out
                
                # Add the heartbeat to the main audio
                ecg_audio_data[start_time:start_time + len(t)] += heartbeat
                pbar.update(1)
        audio_data = ecg_audio_data

    # Normalize and save
    audio_data /= np.max(np.abs(audio_data)) if np.max(np.abs(audio_data)) != 0 else 1
    output_path = f"{output_folder}/{channel_name}_{mystical_name}.wav"
    write(output_path, sample_rate, audio_data.astype(np.float32))
    print(f"Saved {mystical_name} audio for {channel_name} to {output_path}")

# Generate 10% samples (1 distinct sample)
for sample_idx in range(3):
    sample_folder = f"outputs/10_percent_samples/sample_{sample_idx + 1}"
    sampled_data = data.sample(frac=0.10, random_state=sample_idx)
    for band, params in band_to_frequency_and_mode.items():
        for audio_type in ["synth", "wave", "ecg"]:
            generate_audio(
                sampled_data[band],
                params["freq_range"],
                sample_folder,
                audio_type,
                band,
                sample_duration,
            )

# Generate 60-second tracks
for band, params in band_to_frequency_and_mode.items():
    for audio_type in ["synth", "wave", "ecg"]:
        generate_audio(
            data[band].sample(frac=(full_duration / total_duration), random_state=42),
            params["freq_range"],
            "outputs/60_second_tracks",
            audio_type,
            band,
            full_duration,
        )

print("\nAudio files generated successfully!")
